In [0]:
import pandas as pd
import numpy as np
import re
import nltk

# Gathering Data

In [0]:
#reading merged2 csv file which contains different csv files about different products
#for turkish characters using windows-1254 encoder
data = pd.read_csv("merged2.csv", delimiter=";", engine='python', nrows=240000, encoding="windows-1254")

In [3]:
data

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12
0,urls,author,city,ratings,title,comment,product,main_city,part_city,authorName,age,tolower_comment
1,https://www.hepsiburada.com/johnsons-baby-john...,S***** Ş***** (27),MERAM - Konya,width: 60%,ürünün kokusu,fiyatı iyi ama üzgünüm kokusu güzel değil,AILE5148,Konya,MERAM,S*****,27,fiyatı iyi ama üzgünüm kokusu güzel değil
2,https://www.hepsiburada.com/johnsons-baby-john...,tülay üzülmez,HALİLİYE - Şanlıurfa,width: 100%,Güzel,Kısa sürede teslim edildi teşekkürler,AILE5148,Şanlıurfa,HALİLİYE,tülay,NaN,kısa sürede teslim edildi teşekkürler
3,https://www.hepsiburada.com/johnsons-baby-john...,BANU ALTUNBAY,ERGENE - Tekirdağ,width: 100%,Beş Yıldız,"kargo hızlı, ürün iyi ...teşekkürler",AILE5148,Tekirdağ,ERGENE,BANU,NaN,"kargo hızlı, ürün iyi ...teşekkürler"
4,https://www.hepsiburada.com/johnsons-baby-john...,Ergin Dinç (35),BESNİ - Adıyaman,width: 80%,kargo hızlı,kargo hızlı ürünler kutu saglam ezilmemiş.anca...,AILE5148,Adıyaman,BESNİ,Ergin,35,kargo hızlı ürünler kutu saglam ezilmemiş.anca...
...,...,...,...,...,...,...,...,...,...,...,...,...
214505,https://www.hepsiburada.com/komili-islak-havlu...,B***** T***** (37),ÜMRANİYE - İstanbul,width: 100%,Gayet başarılı,Gayet başarılı ürün ve hızlı kargo.,ZYULKER0781002,İstanbul,ÜMRANİYE,B*****,37,gayet başarılı ürün ve hızlı kargo.
214506,https://www.hepsiburada.com/komili-islak-havlu...,İ***** Ç*****,İZMİT - Kocaeli,width: 100%,Beş Yıldız,"Çok hızlı sevkiyat, uygun özel ambalaj",ZYULKER0781002,Kocaeli,İZMİT,İ*****,NaN,"çok hızlı sevkiyat, uygun özel ambalaj"
214507,https://www.hepsiburada.com/komili-islak-havlu...,F***** D*****,ÇANKAYA - Ankara,width: 100%,Beş Yıldız,kaliteli,ZYULKER0781002,Ankara,ÇANKAYA,F*****,NaN,kaliteli
214508,https://www.hepsiburada.com/komili-islak-havlu...,A***** D***** (51),MERKEZ - Eskişehir,width: 60%,Üç Yıldız,doğal bir ürün.kullanışlı,ZYULKER0781002,Eskişehir,MERKEZ,A*****,51,doğal bir ürün.kullanışlı


In [0]:
#to rename column as urls,author,city..
data.columns = data.loc[0].tolist()

In [0]:
#unnecessary now
data = data.drop([214509])

In [0]:
#delete all rows which contains our header urls, author, city...
data.drop(data[data['urls']=='urls'].index, inplace=True)

In [7]:
data.head()

,urls,author,city,ratings,title,comment,product,main_city,part_city,authorName,age,tolower_comment
1,https://www.hepsiburada.com/johnsons-baby-john...,S***** Ş***** (27),MERAM - Konya,width: 60%,ürünün kokusu,fiyatı iyi ama üzgünüm kokusu güzel değil,AILE5148,Konya,MERAM,S*****,27,fiyatı iyi ama üzgünüm kokusu güzel değil
2,https://www.hepsiburada.com/johnsons-baby-john...,tülay üzülmez,HALİLİYE - Şanlıurfa,width: 100%,Güzel,Kısa sürede teslim edildi teşekkürler,AILE5148,Şanlıurfa,HALİLİYE,tülay,NaN,kısa sürede teslim edildi teşekkürler
3,https://www.hepsiburada.com/johnsons-baby-john...,BANU ALTUNBAY,ERGENE - Tekirdağ,width: 100%,Beş Yıldız,"kargo hızlı, ürün iyi ...teşekkürler",AILE5148,Tekirdağ,ERGENE,BANU,NaN,"kargo hızlı, ürün iyi ...teşekkürler"
4,https://www.hepsiburada.com/johnsons-baby-john...,Ergin Dinç (35),BESNİ - Adıyaman,width: 80%,kargo hızlı,kargo hızlı ürünler kutu saglam ezilmemiş.anca...,AILE5148,Adıyaman,BESNİ,Ergin,35,kargo hızlı ürünler kutu saglam ezilmemiş.anca...
5,https://www.hepsiburada.com/johnsons-baby-john...,mehmet koçak (37),KAZAN - Ankara,width: 100%,güzel,tskr ederiz güzel alışveriş oldu...,AILE5148,Ankara,KAZAN,mehmet,37,tskr ederiz güzel alışveriş oldu...


# Data Cleaning

In [8]:
#removing symbols, stopwords
nltk.download('stopwords')
replace_with_space = re.compile('[/(){}\[\]\|@,;]')
remove_symbols1 = re.compile("[^0-9a-z_ğüşıöç .']")
stopwords = nltk.corpus.stopwords.words('turkish')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()
    text = replace_with_space.sub(' ', text)
    text = remove_symbols1.sub('', text)
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
#stopword list
nltk.corpus.stopwords.words('turkish')

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [0]:
#cleaning comments
data.tolower_comment = [clean_text(comment) for comment in data.tolower_comment]

In [11]:
data.tolower_comment

1                     fiyatı iyi üzgünüm kokusu güzel değil
2                     kısa sürede teslim edildi teşekkürler
3                       kargo hızlı ürün iyi ...teşekkürler
4         kargo hızlı ürünler kutu saglam ezilmemiş.anca...
5                       tskr ederiz güzel alışveriş oldu...
                                ...                        
214504    uygun fiyata aldım. kalitesi cidden kötü. boyu...
214505                     gayet başarılı ürün hızlı kargo.
214506                    hızlı sevkiyat uygun özel ambalaj
214507                                             kaliteli
214508                            doğal bir ürün.kullanışlı
Name: tolower_comment, Length: 213661, dtype: object

In [0]:
#to convert percentage ratings to numeric values such as %40 -> 2 -> 1
#after converting numeric values between 1-5
#since it is hard and unnecessary to classsify sentiment from 1 to 5,
#we will classify into 3 segments
#1 will represent negative, 2 Agnostic, 3 Positive

def rating_cleaner(rating):
    int_rating = int("".join(re.findall(r'\d', rating)))//20
    if int_rating==1 or int_rating==2:
        int_rating=1
    elif int_rating==4 or int_rating==5:
        int_rating=3
    else:
        int_rating=2
        
    return int_rating

In [0]:
new_ratings = []
for rating in data["ratings"]:
    new_ratings.append(rating_cleaner(rating))

In [0]:
data['ratings'] = new_ratings

# Data Exploring

In [18]:
data.describe()

,ratings
count,213661.000000
mean,2.916246
std,0.367615
min,1.000000
25%,3.000000
50%,3.000000
75%,3.000000
max,3.000000


In [16]:
data.columns

Index(['urls', 'author', 'city', 'ratings', 'title', 'comment', 'product',
       'main_city', 'part_city', 'authorName', 'age', 'tolower_comment'],
      dtype='object')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213661 entries, 1 to 214508
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   urls             213661 non-null  object
 1   author           213661 non-null  object
 2   city             55658 non-null   object
 3   ratings          213661 non-null  int64 
 4   title            213661 non-null  object
 5   comment          213661 non-null  object
 6   product          213661 non-null  object
 7   main_city        55658 non-null   object
 8   part_city        55658 non-null   object
 9   authorName       213659 non-null  object
 10  age              28866 non-null   object
 11  tolower_comment  213661 non-null  object
dtypes: int64(1), object(11)
memory usage: 21.2+ MB


In [0]:
#most used words
from collections import Counter

most_vocab = Counter()
for comment in data.tolower_comment:
    for word in comment.split(' '):
        if word not in stopwords:
            most_vocab[word] += 1

In [22]:
most_vocab.most_common(10)

[('prima', 114538),
 ('bir', 61818),
 ('bez', 39344),
 ('teşekkürler', 29371),
 ('rahat', 27165),
 ('iyi', 25414),
 ('tavsiye', 25413),
 ('bebeğim', 23394),
 ('ürün', 23295),
 ('cok', 22131)]

# Sentiment Analysis with Classical ML methods

In [38]:
len(data[data['ratings']==1])

6239

In [36]:
len(data[data['ratings']==2])

5417

In [37]:
len(data[data['ratings']==3])

202005

It is clear to see that most of the comments are positive since rating 3 represents the positive reviews.
Fewest rating is Agnostic (2). we want our data equally distributed. So we will take 5000 reviews from each class.

In [0]:
#taking up to 5000 rows from each rating between 1-3
uniform_data = data.groupby('ratings').head(5000)

In [40]:
len(uniform_data)

15000

Count Vectorizer

In [0]:
#It is an option but count vectorizer will not be used in next steps
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
X=vectorizer.fit_transform(uniform_data['tolower_comment'].tolist()).toarray()
y=uniform_data.iloc[: ,3].values

Tf-idf

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm = None, smooth_idf=True)
X=vectorizer.fit_transform(uniform_data['tolower_comment'].tolist()).toarray()
y=uniform_data.iloc[: ,3].values

In [0]:
#splitting train/test (8:2)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

**GaussianNB**

In [70]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
y_pred=classifier.predict(X_test)

In [72]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[817,  89, 111],
       [ 33, 712, 263],
       [ 58,  58, 859]])

In [73]:
print("Vectorizer : Tf-Idf\n"
      "Test/Train size : 0.20\n"
      "Spellchecker usage : No\n"
      "Data Amount : 15,000\n"
      "Accuracy score of GaussianNB :", accuracy_score(y_test, y_pred))

Vectorizer : Tf-Idf
Test/Train size : 0.20
Spellchecker usage : No
Data Amount : 15,000
Accuracy score of GaussianNB : 0.796


In [0]:
def test(classifier, vectorizer):
  comment = ["ürün çok sızdırıyor. berbat",
           "harika. tavsiye ederim",
           "kesinlikle tavsiye etmiyorum",
           "kullanışlı değil.",
           "çocuğumu pişik yaptı",
           "satıcı ilgisiz",
           "başka yerden alın buradan almayın",
           "eksik ve yanlış teslimat",
           "teşekkürler",
           "hızlı kargo",
           "hiç yumuşak değil",
           "iyi",
           "kullanışlı",
           "satıcı çok ilgili",
           "bir daha asla almam çok kötü",
           "tavsiye etmiyorum",
           "çok kötü kokuyor",
           "süper ürün",
           "lastikleri ortalama kalitede", "muhteşem", "harikulade"]
  sample = vectorizer.transform(comment).toarray()
  result = classifier.predict(sample)
  result

  for i in range(len(sample)):
      print(comment[i]+': '+ str(result[i]))

In [83]:
#testing with example sentences
test(classifier, vectorizer)

ürün çok sızdırıyor. berbat: 3
harika. tavsiye ederim: 3
kesinlikle tavsiye etmiyorum: 3
kullanışlı değil.: 3
çocuğumu pişik yaptı: 2
satıcı ilgisiz: 2
başka yerden alın buradan almayın: 2
eksik ve yanlış teslimat: 3
teşekkürler: 3
hızlı kargo: 3
hiç yumuşak değil: 3
iyi: 3
kullanışlı: 3
satıcı çok ilgili: 3
bir daha asla almam çok kötü: 3
tavsiye etmiyorum: 3
çok kötü kokuyor: 3
süper ürün: 3
lastikleri ortalama kalitede: 2
muhteşem: 3
harikulade: 3


Accuracy is 0.79 but as it can be seen in test reviews it is not successful.

**MLPClassfier**

In [55]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


"\nMLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,\n              solver='lbfgs')\n              "

In [0]:
y_pred = clf.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[951,  43,  23],
       [ 59, 831, 118],
       [ 31, 105, 839]])

In [69]:
print("Vectorizer : Tf-Idf\n"
      "Test/Train size : 0.20\n"
      "Spellchecker usage : No\n"
      "Data Amount : 15,000\n"
      "Accuracy score of MLPClassifier :", accuracy_score(y_test, y_pred))

Vectorizer : Tf-Idf
Test/Train size : 0.20
Spellchecker usage : No
Data Amount : 15,000
Accuracy score of MLPClassifier : 0.8736666666666667


In [82]:
#testing with example sentences
test(clf, vectorizer)

ürün çok sızdırıyor. berbat: 1
harika. tavsiye ederim: 3
kesinlikle tavsiye etmiyorum: 1
kullanışlı değil.: 2
çocuğumu pişik yaptı: 1
satıcı ilgisiz: 2
başka yerden alın buradan almayın: 2
eksik ve yanlış teslimat: 1
teşekkürler: 3
hızlı kargo: 3
hiç yumuşak değil: 1
iyi: 3
kullanışlı: 3
satıcı çok ilgili: 3
bir daha asla almam çok kötü: 1
tavsiye etmiyorum: 1
çok kötü kokuyor: 1
süper ürün: 3
lastikleri ortalama kalitede: 3
muhteşem: 3
harikulade: 3


with MLPClassifier we reached higher accuracy than GaussianNB and it seems also successful in test review

**Comparing Logistic Regression, RandomForestClassifier and GaussianNB with Voting Classifier**

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [85]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
  scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.79 (+/- 0.04) [Logistic Regression]
Accuracy: 0.82 (+/- 0.03) [Random Forest]
Accuracy: 0.72 (+/- 0.05) [naive Bayes]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.81 (+/- 0.03) [Ensemble]


In [0]:
'''
Accuracy Results:
Accuracy: 0.79 (+/- 0.04) [Logistic Regression]
Accuracy: 0.82 (+/- 0.03) [Random Forest]
Accuracy: 0.72 (+/- 0.05) [naive Bayes]
'''

In [87]:
#testing random forest classifier which has 0.82 accuracy
clf2.fit(X_train, y_train)
test(clf2, vectorizer)

ürün çok sızdırıyor. berbat: 1
harika. tavsiye ederim: 3
kesinlikle tavsiye etmiyorum: 1
kullanışlı değil.: 1
çocuğumu pişik yaptı: 1
satıcı ilgisiz: 3
başka yerden alın buradan almayın: 1
eksik ve yanlış teslimat: 1
teşekkürler: 3
hızlı kargo: 3
hiç yumuşak değil: 1
iyi: 3
kullanışlı: 3
satıcı çok ilgili: 3
bir daha asla almam çok kötü: 1
tavsiye etmiyorum: 1
çok kötü kokuyor: 1
süper ürün: 3
lastikleri ortalama kalitede: 3
muhteşem: 3
harikulade: 3


**SVM**

In [20]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(X_test)

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[957,  21,  39],
       [ 56, 800, 152],
       [ 47,  39, 889]])

In [28]:
print("Vectorizer : Tf-Idf\n"
      "Test/Train size : 0.20\n"
      "Spellchecker usage : No\n"
      "Data Amount : 15,000\n"
      "Accuracy score of SVM :", accuracy_score(y_test, y_pred))

Vectorizer : Tf-Idf
Test/Train size : 0.20
Spellchecker usage : No
Data Amount : 15,000
Accuracy score of SVM : 0.882


So far we reached highest accuracy with SVM

# Spell Check

Since this data is taken from hepsiburada.com the comment part is little dirty. This is because comments are not checked by any moderator. There might be many misspelled words.
Therefore we will apply turkishnlp library to check spelling and compare results.

In [13]:
!pip install turkishnlp

  Created wheel for turkishnlp: filename=turkishnlp-0.0.61-cp36-none-any.whl size=7557 sha256=bbbd4a27aa6a3cd4a7a468eb2f66d9d6e1105624f095a11e878532a9a5bda1a6
  Stored in directory: /root/.cache/pip/wheels/06/11/95/9c04635b9b6cb20e8036e26a6cadaa86b32521a85155b6f1ca
Successfully built turkishnlp


In [0]:
import turkishnlp

In [0]:
from turkishnlp import detector
obj = detector.TurkishNLP()

In [31]:
obj.download()

Download is successful


In [0]:
obj.create_word_set()

In [41]:
#spellcheck for every word in comments
uniform_data['corrected_comment'] = [" ".join(obj.auto_correct(obj.list_words(comment))) for comment in uniform_data['tolower_comment']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
uniform_data

,urls,author,city,ratings,title,comment,product,main_city,part_city,authorName,age,tolower_comment,corrected_comment
1,https://www.hepsiburada.com/johnsons-baby-john...,S***** Ş***** (27),MERAM - Konya,2,ürünün kokusu,fiyatı iyi ama üzgünüm kokusu güzel değil,AILE5148,Konya,MERAM,S*****,27,fiyatı iyi üzgünüm kokusu güzel değil,fiyatı iyi üzgünüm kokusu güzel değil
2,https://www.hepsiburada.com/johnsons-baby-john...,tülay üzülmez,HALİLİYE - Şanlıurfa,3,Güzel,Kısa sürede teslim edildi teşekkürler,AILE5148,Şanlıurfa,HALİLİYE,tülay,NaN,kısa sürede teslim edildi teşekkürler,kısa sürede teslim edildi teşekkürler
3,https://www.hepsiburada.com/johnsons-baby-john...,BANU ALTUNBAY,ERGENE - Tekirdağ,3,Beş Yıldız,"kargo hızlı, ürün iyi ...teşekkürler",AILE5148,Tekirdağ,ERGENE,BANU,NaN,kargo hızlı ürün iyi ...teşekkürler,kargo hızlı ürün iyi teşekkürler
4,https://www.hepsiburada.com/johnsons-baby-john...,Ergin Dinç (35),BESNİ - Adıyaman,3,kargo hızlı,kargo hızlı ürünler kutu saglam ezilmemiş.anca...,AILE5148,Adıyaman,BESNİ,Ergin,35,kargo hızlı ürünler kutu saglam ezilmemiş.anca...,kargo hızlı ürünler kutu saglam ezilmemiş anca...
5,https://www.hepsiburada.com/johnsons-baby-john...,mehmet koçak (37),KAZAN - Ankara,3,güzel,tskr ederiz güzel alışveriş oldu...,AILE5148,Ankara,KAZAN,mehmet,37,tskr ederiz güzel alışveriş oldu...,tsk ederiz güzel alışveriş oldu
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202223,https://www.hepsiburada.com/prima-kulot-bebek-...,Deniz Cingöz,NaN,2,Üç Yıldız,Ürünün teslimi ile ilgili her şey çok iyiydi a...,ZYPYON74023,NaN,NaN,Deniz,NaN,ürünün teslimi ilgili iyiydi ancak ürün 611 ki...,ürünün teslimi ilgili iyiydi ancak ürün kilo o...
202235,https://www.hepsiburada.com/prima-kulot-bebek-...,Bu yorum Prima.com.tr websitesinden alınmıştır.,NaN,2,Primaaa,Çok iyi bir ürün çok severek kullanıyoruz teşe...,ZYPYON74023,NaN,NaN,Bu,NaN,iyi bir ürün severek kullanıyoruz teşekkürler ...,iyi bir ürün severek kullanıyoruz teşekkürler ...
202238,https://www.hepsiburada.com/prima-kulot-bebek-...,Bu yorum Prima.com.tr websitesinden alınmıştır.,NaN,2,Primaa,Çok memnun kaldim herkese tavsiyemdir. Çocugum...,ZYPYON74023,NaN,NaN,Bu,NaN,memnun kaldim herkese tavsiyemdir. çocugumda z...,memnun kaldim herkese tavsiyemdir çocuğunda zm...
202393,https://www.hepsiburada.com/prima-kulot-bebek-...,Bu yorum Prima.com.tr websitesinden alınmıştır.,NaN,2,prima külot bez,Bebekler emeklemeye başladıktan sonra bezlerin...,ZYPYON74023,NaN,NaN,Bu,NaN,bebekler emeklemeye başladıktan sonra bezlerin...,bebekler emeklemeye başladıktan sonra bezlerin...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm = None, smooth_idf=True)
X=vectorizer.fit_transform(uniform_data['corrected_comment'].tolist()).toarray()
y=uniform_data.iloc[: ,3].values

In [0]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred=classifier.predict(X_test)

In [57]:
print("Vectorizer : Tf-Idf\n"
      "Test/Train size : 0.20\n"
      "Spellchecker usage : Yes\n"
      "Data Amount : 15,000\n"
      "Accuracy score of GaussianNB :", accuracy_score(y_test, y_pred))

Vectorizer : Tf-Idf
Test/Train size : 0.20
Spellchecker usage : Yes
Data Amount : 15,000
Accuracy score of GaussianNB : 0.7943333333333333


In [0]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)

In [0]:
classifier.fit(X_train, y_train)
y_pred=classifier.predict(X_test)

In [60]:
print("Vectorizer : Tf-Idf\n"
      "Test/Train size : 0.20\n"
      "Spellchecker usage : Yes\n"
      "Data Amount : 15,000\n"
      "Accuracy score of RandomForestClassifier :", accuracy_score(y_test, y_pred))

Vectorizer : Tf-Idf
Test/Train size : 0.20
Spellchecker usage : Yes
Data Amount : 15,000
Accuracy score of RandomForestClassifier : 0.7943333333333333


We can see that nearly nothing changed. even we can say that we reached less accuracy.

# Word2Vec + LSTM

In [0]:
#Word2Vec approach for sentiment analysis
from gensim import utils
from gensim.models import word2vec

In [0]:
from sklearn.model_selection import train_test_split
X=uniform_data['tolower_comment'].tolist()
y=uniform_data.iloc[: ,3].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [0]:
n_dim = 300
#Initialize model and build vocab
model = word2vec.Word2Vec(size=n_dim, min_count=1, window=10, iter=5)
model.build_vocab(map(lambda x: x.split(), X_train))

In [55]:
model.train(map(lambda x: x.split(), X_train), total_examples=model.corpus_count, epochs=model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(186983, 200855)

In [0]:
#exploring word and vectors
words = list(model.wv.vocab)

In [52]:
len(words)

16683

In [51]:
words

['ürün',
 'iyi.',
 'fiyat',
 'güzel',
 'fakat',
 'ptt',
 'kargo',
 'tam',
 'bir',
 'rezalet.',
 'ekonomik',
 'hızlı',
 'gonderim.',
 'tesekkurler...',
 'beğendim',
 '5aydir',
 'kullanıyorum',
 'memnunum',
 'kaliteli',
 'geldi',
 'üstelik',
 'beklediğimden',
 'teslim',
 'edildi',
 'teşekkürler...',
 'süper',
 'teşekkürler',
 'hb.',
 'oglum',
 'suan',
 '16',
 'aylik',
 'dogdugundan',
 'beri',
 'prima',
 'kullandik.ama',
 'su',
 'emici',
 'kanal',
 'berbat',
 'son',
 '2',
 'paket',
 'gece',
 'cisi',
 'mutlaka',
 'sizdiriyor.uyku',
 'arasi',
 '4',
 'saatte',
 'bez',
 'degistiriyorm',
 'yinede',
 'zibin',
 'pijama',
 'tulum',
 'varsa',
 'islak.havalar',
 'sogudu',
 'cocuklar',
 'hasta',
 'olucak',
 'herkes',
 'ayni',
 'seyden',
 'sikayetci.sorun',
 'bizdemi',
 'dedim',
 'gittim',
 'komili',
 'aldim',
 'gecedir',
 '12',
 'saate',
 'yakin',
 'alntinda',
 'hic',
 'sizdirma',
 'yok',
 'demekki',
 'sorun',
 'sizde',
 'yeni',
 'biseylere',
 'bizce',
 'gerek',
 'yokmus',
 'adet',
 'eksiği',
 'var'

In [57]:
model.most_similar("ürün")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gayet', 0.9999841451644897),
 ('geldi', 0.9999815225601196),
 ('kaliteli', 0.9999787211418152),
 ('kargo', 0.9999732971191406),
 ('ptt', 0.999969482421875),
 ('elime', 0.9999690055847168),
 ('güzel', 0.9999687671661377),
 ('hepsiburada', 0.999967634677887),
 ('uygun', 0.9999663233757019),
 ('teşekkürler', 0.9999662637710571)]

In [49]:
np.average([model[w] for w in X_train[0].split() if w in model.wv.vocab], axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 5.09234727e-04, -2.63399590e-04, -8.07167453e-05, -1.60348689e-04,
        1.69916806e-04,  7.07114697e-04,  7.24066058e-06, -8.22991115e-06,
        2.98160303e-04,  1.67676873e-04, -2.40619716e-04,  1.79697920e-04,
       -2.23439376e-04, -2.63250840e-04,  4.19445481e-04, -4.26930055e-04,
        2.97264982e-04,  6.88946966e-05,  9.18267033e-05, -6.09365408e-04,
        3.43707652e-04,  4.05677347e-05,  1.07504216e-04,  4.10202301e-05,
       -2.11961946e-04, -2.10875689e-04, -3.47328692e-04,  1.63359131e-04,
        3.32132040e-05,  6.13648467e-07,  4.28168394e-04,  5.36682666e-04,
        3.16499179e-04, -3.86529136e-05,  3.39448452e-04,  3.27782996e-04,
        2.77194304e-05,  2.65619106e-04, -1.44307109e-04,  3.96096788e-04,
        1.39345910e-04, -1.25423801e-04, -1.96111709e-04,  7.67185993e-04,
       -7.02473393e-04,  2.75834551e-04,  3.53441748e-04, -3.79184676e-05,
        7.48948834e-04, -2.64402825e-05,  2.83621630e-04, -5.72377816e-04,
        4.88302321e-04, -

In [61]:
#analogy for iyi-kötü = hızlı-yavaş
model.most_similar(np.array([model["iyi"]-model["kötü"]+model["yavaş"]]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hızlı', 0.9997151494026184),
 ('uygun', 0.9996981620788574),
 ('güzel', 0.999687671661377),
 ('teslimat', 0.9996804594993591),
 ('kargo', 0.9996774196624756),
 ('teşekkür', 0.9996749758720398),
 ('teşekkürler', 0.9996466636657715),
 ('fiyat', 0.9996416568756104),
 ('paketleme', 0.999631404876709),
 ('ederim', 0.9996182918548584)]

In [64]:
model.most_similar(np.array([model["hızlı"]-model["teslimat"]+model["iyi"]]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gayet', 0.9999761581420898),
 ('ürün', 0.9999748468399048),
 ('kaliteli', 0.9999744296073914),
 ('geldi', 0.9999687671661377),
 ('hepsiburada', 0.9999686479568481),
 ('fiyata', 0.999967634677887),
 ('fiyatı', 0.9999669790267944),
 ('iyi', 0.9999622106552124),
 ('geç', 0.9999605417251587),
 ('ptt', 0.9999601244926453)]

In [0]:
#helper source: https://github.com/ninadakolekar/product-review-sentiment-analysis/blob/master/SentimentAnalysis.ipynb
def transformedFeatures(review, w2v_model, embed_size=300):

    vocabulary = set(w2v_model.wv.index2word)
    feat = np.zeros((embed_size,),dtype=float)

    count = 0
    flag = True
    for word in review:

        if word in vocabulary: 

            count = count + 1.
            feat = np.add(feat, w2v_model[word])
            flag = False

    if flag == False:
        feat = np.divide(feat, count)

    return feat


def transformReviews(reviews, model, embed_size):

    num_reviews = len(reviews)
    
    transformedReviewFeat = np.zeros((num_reviews,embed_size),dtype=float)
    for idx, review in enumerate(reviews):
        transformedReviewFeat[idx] = transformedFeatures(review, model,embed_size)

    return transformedReviewFeat

In [74]:
train_vectors = transformReviews([x.split() for x in X_train], model, 300)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [75]:
test_vectors = transformReviews([x.split() for x in X_test], model, 300)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [77]:
#expected shape is (300,) because vector size is determined as 300
model["ürün"].shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [76]:
#expected shape (16683,300) because 16683 words used and vector size is determined as 300
embedding_matrix = np.array(model.wv.syn0)
print("Shape of embedding matrix : ", embedding_matrix.shape)

Shape of embedding matrix :  (16683, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [78]:
embedding_matrix.shape[0]

16683

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, Activation, Embedding
from keras.utils import np_utils

In [97]:
# Fit tokenizer on training set and then transform training and test reviews

tokenizer = Tokenizer(nb_words=embedding_matrix.shape[0])
tokenizer.fit_on_texts(X_train)
train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq = sequence.pad_sequences(train_seq, maxlen=100)
X_test_seq = sequence.pad_sequences(test_seq, maxlen=100)
y_train_seq = np_utils.to_categorical(y_train, 4)
y_test_seq = np_utils.to_categorical(y_test, 4)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [99]:
model2 = Sequential(
    [Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix]),
    Bidirectional(LSTM(128, dropout_W=0.2, dropout_U=0.2)),
    Dense(4),
    Activation('softmax')]
)

model2.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         5004900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1028      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 5,445,224
Trainable params: 5,445,224
Non-trainable params: 0
_________________________________________________________________


In [100]:
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.fit(X_train_seq, y_train_seq, batch_size=32, epochs=3, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
12000/12000 [==============================] - 195s 16ms/step - loss: 0.3396 - accuracy: 0.8363
Epoch 2/3
12000/12000 [==============================] - 194s 16ms/step - loss: 0.1276 - accuracy: 0.9519
Epoch 3/3
12000/12000 [==============================] - 194s 16ms/step - loss: 0.0632 - accuracy: 0.9784


In [102]:
# Evaluate trained model
_, test_accuracy = model2.evaluate(X_test_seq, y_test_seq, batch_size=32)
print(f'Test accuracy : {round(test_accuracy,2)}')

3000/3000 [==============================] - 10s 3ms/step
Test accuracy : 0.94


In [0]:
# Save model architecture by serializing to JSON
json_model = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(json_model)

# Save model weights by serializing to HDF5
model2.save_weights("model2.h5")

In [0]:
def test2(reviews):
  review_seq = tokenizer.texts_to_sequences(reviews)
  review_seq = sequence.pad_sequences(review_seq, maxlen=100)

  proba = model2.predict_proba(review_seq)

  sentiments = []
  for x in proba.tolist():
    class_idx = np.argmax(x)
    if class_idx==1:
      sentiment="Negative"
    elif class_idx==3:
      sentiment="Positive"
    elif class_idx==2:
      sentiment="Agnostic"
    class_proba = x[class_idx]
    sentiments.append((sentiment,class_proba))

  return sentiments

In [0]:
  reviews = ["ürün çok sızdırıyor. berbat",
           "harika. tavsiye ederim",
           "kesinlikle tavsiye etmiyorum",
           "kullanışlı değil.",
           "çocuğumu pişik yaptı",
           "satıcı ilgisiz",
           "başka yerden alın buradan almayın",
           "eksik ve yanlış teslimat",
           "teşekkürler",
           "hızlı kargo",
           "hiç yumuşak değil",
           "iyi",
           "kullanışlı",
           "satıcı çok ilgili",
           "bir daha asla almam çok kötü",
           "tavsiye etmiyorum",
           "çok kötü kokuyor",
           "süper ürün",
           "lastikleri ortalama kalitede", "muhteşem", "harikulade"]

In [116]:
for i in range(len(reviews)):
  print(reviews[i],test2(reviews)[i])

ürün çok sızdırıyor. berbat ('Negative', 0.9641702175140381)
harika. tavsiye ederim ('Positive', 0.8762253522872925)
kesinlikle tavsiye etmiyorum ('Negative', 0.9460793137550354)
kullanışlı değil. ('Agnostic', 0.4900384843349457)
çocuğumu pişik yaptı ('Negative', 0.9368640780448914)
satıcı ilgisiz ('Agnostic', 0.9198565483093262)
başka yerden alın buradan almayın ('Negative', 0.9829283952713013)
eksik ve yanlış teslimat ('Negative', 0.906998336315155)
teşekkürler ('Positive', 0.8548956513404846)
hızlı kargo ('Positive', 0.7566068172454834)
hiç yumuşak değil ('Negative', 0.5716693997383118)
iyi ('Positive', 0.7326992750167847)
kullanışlı ('Positive', 0.5028775334358215)
satıcı çok ilgili ('Positive', 0.5132745504379272)
bir daha asla almam çok kötü ('Negative', 0.9560171961784363)
tavsiye etmiyorum ('Negative', 0.9360243678092957)
çok kötü kokuyor ('Negative', 0.49065542221069336)
süper ürün ('Positive', 0.7800518870353699)
lastikleri ortalama kalitede ('Negative', 0.874934732913971)
mu

When all methods compared we see that word2vec+lstm approach is clearly the winner with 0.94 accuracy.